# Using deep features to build an image classifier

# Fire up GraphLab Create
(See [Getting Started with SFrames](../Week%201/Getting%20Started%20with%20SFrames.ipynb) for setup instructions)

In [1]:
import graphlab
import os

In [2]:
# Set product key on this computer. After running this cell, you will not need to re-enter your product key.
# Here the product key is stored in an environment variable. Can just be entered as a string.
graphlab.product_key.set_product_key(os.environ.get('GraphlabCreate'))

# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

# Output active product key.
# graphlab.product_key.get_product_key()

This non-commercial license of GraphLab Create for academic use is assigned to andreas.keller@gmail.com and will expire on December 19, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1487583884.log


# Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [3]:
image_train = graphlab.SFrame('data/image_train_data/')
image_test = graphlab.SFrame('data/image_test_data/')

# Exploring the image data

In [4]:
# graphlab.canvas.set_target('ipynb')

In [5]:
image_train['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:49664/index.html
Opening Canvas in default web browser.


# Train a classifier on the raw image pixels

We first start by training a classifier on just the raw pixels of the image.

In [6]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

# Make a prediction with the simple model based on raw pixels

In [7]:
image_test[0:3]['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:49664/index.html
Opening Canvas in default web browser.


Logistic regression:

--------------------------------------------------------

Number of examples          : 1891

Number of classes           : 4

...

| 6         | 12       | 1.000000  | 7.011182     | 0.454257          | 0.385965            |

| 10        | 17       | 1.000000  | 10.566657    | 0.513485          | 0.491228            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [8]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [9]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

The model makes wrong predictions for all three images.

# Evaluating raw pixel model on test data

In [10]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.47325, 'auc': 0.7153652499999986, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       bird      |   97  |
 |     bird     |       bird      |  500  |
 |     bird     |       dog       |  172  |
 |     dog      |       dog       |  407  |
 |     cat      |       dog       |  279  |
 |     dog      |    automobile   |  126  |
 |     bird     |       cat       |  167  |
 |     cat      |       cat       |  328  |
 |     cat      |    automobile   |  186  |
 |     dog      |       bird      |  231  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.4686240691499787, 'log_loss': 1.2195687750477977, 'precision': 0

The accuracy of this model is poor, getting only about 46% accuracy.

# Can we improve the model using deep features

We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data.  Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [11]:
len(image_train)

2005

## Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [12]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

As we can see, the column deep_features already contains the pre-computed deep features for this data. 

In [13]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features, let's train a classifier

In [14]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

--------------------------------------------------------

+-----------+----------+-----------+--------------+-------------------+---------------------+

| Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |

+-----------+----------+-----------+--------------+-------------------+---------------------+

| 1         | 5        | 0.000131  | 2.744378     | 0.698896          | 0.647059            |

# Apply the deep features model to first few images of test set

In [15]:
image_test[0:3]['image'].show()

| 2         | 9        | 0.250000  | 5.289905     | 0.765633          | 0.803922            |

| 3         | 10       | 0.250000  | 6.056562     | 0.769837          | 0.803922            |

Canvas is updated and available in a tab in the default browser.


| 4         | 11       | 0.250000  | 6.783326     | 0.778245          | 0.813725            |

| 5         | 12       | 0.250000  | 7.526815     | 0.794535          | 0.813725            |

In [16]:
deep_features_model.predict(image_test[0:3])

| 6         | 13       | 0.250000  | 8.375958     | 0.810825          | 0.813725            |

| 7         | 14       | 0.250000  | 9.187861     | 0.822386          | 0.794118            |

| 8         | 15       | 0.250000  | 10.056793    | 0.831844          | 0.784314            |

| 9         | 16       | 0.250000  | 10.864566    | 0.847084          | 0.813725            |

| 10        | 17       | 0.250000  | 11.769823    | 0.856017          | 0.823529            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

The classifier with deep features gets all of these images right!

# Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [17]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.7765, 'auc': 0.9381719999999976, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       cat       |   9   |
 |     bird     |       dog       |   75  |
 |     cat      |       bird      |   78  |
 |  automobile  |       dog       |   7   |
 |     cat      |    automobile   |   50  |
 |     dog      |       bird      |   45  |
 |     bird     |       cat       |   97  |
 |     dog      |    automobile   |   21  |
 |     dog      |       dog       |  761  |
 |     cat      |       dog       |  287  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7740968088844067, 'log_loss': 0.5976938037822069, 'precision': 0.

# Assignments
## Assignment 1

Summary statistics of the data

In [18]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


## Assignment 2
Split data on each category

In [19]:
image_train_automobile = image_train[image_train['label'] == 'automobile']
image_train_cat = image_train[image_train['label'] == 'cat']
image_train_dog = image_train[image_train['label'] == 'dog']
image_train_bird = image_train[image_train['label'] == 'bird']

Train model for each category

In [20]:
automobile_model = graphlab.nearest_neighbors.create(image_train_automobile,
                                                     features = ['deep_features'],
                                                     label='id')
cat_model = graphlab.nearest_neighbors.create(image_train_cat,
                                              features = ['deep_features'],
                                              label='id')
dog_model = graphlab.nearest_neighbors.create(image_train_dog,
                                              features = ['deep_features'],
                                              label='id')
bird_model = graphlab.nearest_neighbors.create(image_train_bird,
                                               features = ['deep_features'],
                                               label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Consider cat image

In [21]:
image_test[0:1]['image'].show()

Canvas is updated and available in a tab in the default browser.


Nearest cat in the training data

In [22]:
cat = cat_model.query(image_test[0:1])
cat

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 35.839ms     |

| Done         |         | 100         | 121.698ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [23]:
image_train_cat[image_train_cat['id'] == 16289]['image'].show()

Canvas is updated and available in a tab in the default browser.


Nearest dog in the training data

In [24]:
dog = dog_model.query(image_test[0:1])
dog

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 35.024ms     |

| Done         |         | 100         | 111.749ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [25]:
image_train_dog[image_train_dog['id'] == 16976]['image'].show()

Canvas is updated and available in a tab in the default browser.


Define function to retrieve images

## Assignment 3
Mean distance of cats

In [26]:
cat['distance'].mean()

36.15573070978294

Mean distance of dog

In [27]:
dog['distance'].mean()

37.77071136184156

## Assignment 4
Split test data

In [28]:
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_bird = image_test[image_test['label'] == 'bird']
image_test_automobile = image_test[image_test['label'] == 'automobile']

Finding nearest neighbors in the training set for each part of the test set

In [29]:
# Dog test images
dog_dog_neighbors = dog_model.query(image_test_dog, k = 1)
dog_cat_neighbors = cat_model.query(image_test_dog, k = 1)
dog_bird_neighbors = bird_model.query(image_test_dog, k = 1)
dog_automobile_neighbors = automobile_model.query(image_test_dog, k = 1)

# Cat test images
cat_cat_neighbors = cat_model.query(image_test_cat, k = 1)
cat_dog_neighbors = dog_model.query(image_test_cat, k = 1)
cat_bird_neighbors = bird_model.query(image_test_cat, k = 1)
cat_automibole_neighbors = automobile_model.query(image_test_cat, k = 1)

# Bird test images
bird_bird_neighbors = bird_model.query(image_test_bird, k = 1)
bird_dog_neighbors = dog_model.query(image_test_bird, k = 1)
bird_cat_neighbors = cat_model.query(image_test_bird, k = 1)
bird_automobile_neighbors = automobile_model.query(image_test_bird, k = 1)

# Automobile test images
automobile_automobile_neighbors = automobile_model.query(image_test_automobile, k = 1)
automobile_dog_neighbors = dog_model.query(image_test_automobile, k = 1)
automobile_cat_neighbors = cat_model.query(image_test_automobile, k = 1)
automobile_bird_neighbors = bird_model.query(image_test_automobile, k = 1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 120000  | 25.1046     | 476.484ms    |

| Done         | 478000  | 100         | 519.214ms    |

+--------------+---------+-------------+--------------+

In [30]:
dog_distances = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],
                                 'dog-bird': dog_bird_neighbors['distance'],
                                 'dog-cat': dog_cat_neighbors['distance'],
                                 'dog-dog': dog_dog_neighbors['distance']})

In [31]:
dog_distances.head()

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


## Assignment 4

Computing the number of correct predictions using 1-nearest neighbors for the dog class



In [32]:
def is_dog_correct(row):
    if min(row, key = row.get) == 'dog-dog':
        return 1
    else:
        return 0

In [33]:
dog_distances.apply(is_dog_correct).sum()

678

Accuracy

In [34]:
dog_distances.apply(is_dog_correct).sum() / float(dog_distances.num_rows())

0.678